In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import cohen_kappa_score, log_loss
from sklearn.utils.class_weight import compute_class_weight
import itertools
import matplotlib.pyplot as plt
#from tqdm.notebook import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import random
import os
import math
import pickle

In [ ]:
from scipy.stats import rankdata

def greedy_predict(outputs, train_labels):
    rank = np.round(rankdata(np.array(outputs))/len(outputs), decimals=5)
    train_class_cumsum_ratio = np.round(np.cumsum(np.bincount(np.array(train_labels))/len(train_labels)), decimals=5)
    
    pred = rank.copy()
    for i, threshould in reversed(list(enumerate(train_class_cumsum_ratio[:-1]))):
        pred = np.where((pred > threshould) & (pred < i+1), i+1, pred)
    return pred.astype(int).tolist()

In [ ]:
from functools import partial
from scipy import optimize

class OptimizedRounder(object):
    def __init__(self,
                 n_classes: int = 5,
                 initial_coef = [i+0.5 for i in range(5)]):
        self.coef_ = 0
        self.n_classes = n_classes
        self.initial_coef = initial_coef

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            X_p[i] = self.n_classes - 1 
            for j in range(self.n_classes-1):
                if pred < coef[j]:
                    X_p[i] = j
                    break
        ll = cohen_kappa_score(y, X_p, labels=None, weights='quadratic', sample_weight=None)
        return -ll
    
    def _CEM_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            X_p[i] = self.n_classes - 1 
            for j in range(self.n_classes-1):
                if pred < coef[j]:
                    X_p[i] = j
                    break
        ll = CEM(y, X_p.astype(int))
        return -ll

    def fit(self, X, y, loss_func = 'qwk'):
        if loss_func == 'qwk':
            loss_partial = partial(self._kappa_loss, X=X, y=y)
        elif loss_func == 'cem':
            loss_partial = partial(self._CEM_loss, X=X, y=y)
        else:
            raise RuntimeError('loss function does not exist!')
        self.coef_ = optimize.minimize(loss_partial, self.initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            X_p[i] = self.n_classes - 1 
            for j in range(self.n_classes-1):
                if pred < coef[j]:
                    X_p[i] = j
                    break
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [ ]:
def CIQ(label_num):
    return [[-math.log2((label_num[i]/2+sum(label_num[i+1:j+1]))/sum(label_num)) if i<=j 
             else -math.log2((label_num[i]/2+sum(label_num[j:i]))/sum(label_num))
             for j in range(len(label_num))] for i in range(len(label_num))]

def CEM(all_labels, all_preds):
    prox = CIQ(np.bincount(all_labels))
    return sum([prox[pred][label] for label, pred in zip(all_labels, all_preds)])/sum([prox[label][label] for label in all_labels])

def calc_PCNL(labels_count):
    return [(sum(labels_count[0:i])+sum(labels_count[1:i+1]))/(2*sum(labels_count)-labels_count[0]-labels_count[len(labels_count)-1])
           for i in range(len(labels_count))] 

In [ ]:
import pickle
f = open('multiclass_logloss_losses.txt', 'rb')
multiclass_logloss_losses = pickle.load(f)
f.close()
f = open('multiclass_logloss_QWKs.txt', 'rb')
multiclass_logloss_QWKs = pickle.load(f)
f.close()
f = open('multiclass_logloss_CEMs.txt', 'rb')
multiclass_logloss_CEMs = pickle.load(f)
f.close()
f = open('multiclass_logloss_valid_preds.txt', 'rb')
multiclass_logloss_valid_preds = pickle.load(f)
f.close()

In [ ]:
f = open('reg_class_OWAE_losses.txt', 'rb')
reg_class_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_class_OWAE_QWK.txt', 'rb')
reg_class_OWAE_QWKs = pickle.load(f)
f.close()
f = open('reg_class_OWAE_QWK2.txt', 'rb')
reg_class_OWAE_QWK2s = pickle.load(f)
f.close()
f = open('reg_class_OWAE_QWK3.txt', 'rb')
reg_class_OWAE_QWK3s = pickle.load(f)
f.close()
f = open('reg_class_OWAE_CEM.txt', 'rb')
reg_class_OWAE_CEMs = pickle.load(f)
f.close()
f = open('reg_class_OWAE_CEM2.txt', 'rb')
reg_class_OWAE_CEM2s = pickle.load(f)
f.close()
f = open('reg_class_OWAE_CEM3.txt', 'rb')
reg_class_OWAE_CEM3s = pickle.load(f)
f.close()
f = open('reg_class_OWAE_valid_preds.txt', 'rb')
reg_class_OWAE_valid_preds = pickle.load(f)
f.close()
f = open('reg_class_OWAE_valid_outputs.txt', 'rb')
reg_class_OWAE_valid_outputs = pickle.load(f)
f.close()

In [ ]:
f = open('reg_class_OWSE_losses.txt', 'rb')
reg_class_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_class_OWSE_QWK.txt', 'rb')
reg_class_OWSE_QWKs = pickle.load(f)
f.close()
f = open('reg_class_OWSE_QWK2.txt', 'rb')
reg_class_OWSE_QWK2s = pickle.load(f)
f.close()
f = open('reg_class_OWSE_QWK3.txt', 'rb')
reg_class_OWSE_QWK3s = pickle.load(f)
f.close()
f = open('reg_class_OWSE_CEM.txt', 'rb')
reg_class_OWSE_CEMs = pickle.load(f)
f.close()
f = open('reg_class_OWSE_CEM2.txt', 'rb')
reg_class_OWSE_CEM2s = pickle.load(f)
f.close()
f = open('reg_class_OWSE_CEM3.txt', 'rb')
reg_class_OWSE_CEM3s = pickle.load(f)
f.close()
f = open('reg_class_OWSE_valid_preds.txt', 'rb')
reg_class_OWSE_valid_preds = pickle.load(f)
f.close()
f = open('reg_class_OWSE_valid_outputs.txt', 'rb')
reg_class_OWSE_valid_outputs = pickle.load(f)
f.close()

In [ ]:
f = open('reg_PCNL_OWAE_losses.txt', 'rb')
reg_PCNL_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_QWK.txt', 'rb')
reg_PCNL_OWAE_QWKs = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_QWK2.txt', 'rb')
reg_PCNL_OWAE_QWK2s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_QWK3.txt', 'rb')
reg_PCNL_OWAE_QWK3s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_CEM.txt', 'rb')
reg_PCNL_OWAE_CEMs = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_CEM2.txt', 'rb')
reg_PCNL_OWAE_CEM2s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_CEM3.txt', 'rb')
reg_PCNL_OWAE_CEM3s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_valid_preds.txt', 'rb')
reg_PCNL_OWAE_valid_preds = pickle.load(f)
f.close()
f = open('reg_PCNL_OWAE_valid_outputs_PCNL.txt', 'rb')
reg_PCNL_OWAE_valid_outputs_PCNL = pickle.load(f)
f.close()

In [ ]:
f = open('reg_PCNL_OWSE_losses.txt', 'rb')
reg_PCNL_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_QWK.txt', 'rb')
reg_PCNL_OWSE_QWKs = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_QWK2.txt', 'rb')
reg_PCNL_OWSE_QWK2s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_QWK3.txt', 'rb')
reg_PCNL_OWSE_QWK3s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_CEM.txt', 'rb')
reg_PCNL_OWSE_CEMs = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_CEM2.txt', 'rb')
reg_PCNL_OWSE_CEM2s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_CEM3.txt', 'rb')
reg_PCNL_OWSE_CEM3s = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_valid_preds.txt', 'rb')
reg_PCNL_OWSE_valid_preds = pickle.load(f)
f.close()
f = open('reg_PCNL_OWSE_valid_outputs_PCNL.txt', 'rb')
reg_PCNL_OWSE_valid_outputs_PCNL = pickle.load(f)
f.close()

In [ ]:
f = open('reg_class_MAE_losses.txt', 'rb')
reg_class_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_class_MAE_QWK.txt', 'rb')
reg_class_MAE_QWKs = pickle.load(f)
f.close()
f = open('reg_class_MAE_QWK2.txt', 'rb')
reg_class_MAE_QWK2s = pickle.load(f)
f.close()
f = open('reg_class_MAE_QWK3.txt', 'rb')
reg_class_MAE_QWK3s = pickle.load(f)
f.close()
f = open('reg_class_MAE_CEM.txt', 'rb')
reg_class_MAE_CEMs = pickle.load(f)
f.close()
f = open('reg_class_MAE_CEM2.txt', 'rb')
reg_class_MAE_CEM2s = pickle.load(f)
f.close()
f = open('reg_class_MAE_CEM3.txt', 'rb')
reg_class_MAE_CEM3s = pickle.load(f)
f.close()
f = open('reg_class_MAE_valid_preds.txt', 'rb')
reg_class_MAE_valid_preds = pickle.load(f)
f.close()
f = open('reg_class_MAE_valid_outputs.txt', 'rb')
reg_class_MAE_valid_outputs = pickle.load(f)
f.close()
f = open('reg_class_MSE_losses.txt', 'rb')
reg_class_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_class_MSE_QWK.txt', 'rb')
reg_class_MSE_QWKs = pickle.load(f)
f.close()
f = open('reg_class_MSE_QWK2.txt', 'rb')
reg_class_MSE_QWK2s = pickle.load(f)
f.close()
f = open('reg_class_MSE_QWK3.txt', 'rb')
reg_class_MSE_QWK3s = pickle.load(f)
f.close()
f = open('reg_class_MSE_CEM.txt', 'rb')
reg_class_MSE_CEMs = pickle.load(f)
f.close()
f = open('reg_class_MSE_CEM2.txt', 'rb')
reg_class_MSE_CEM2s = pickle.load(f)
f.close()
f = open('reg_class_MSE_CEM3.txt', 'rb')
reg_class_MSE_CEM3s = pickle.load(f)
f.close()
f = open('reg_class_MSE_valid_preds.txt', 'rb')
reg_class_MSE_valid_preds = pickle.load(f)
f.close()
f = open('reg_class_MSE_valid_outputs.txt', 'rb')
reg_class_MSE_valid_outputs = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_losses.txt', 'rb')
reg_PCNL_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_QWK.txt', 'rb')
reg_PCNL_MAE_QWKs = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_QWK2.txt', 'rb')
reg_PCNL_MAE_QWK2s = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_QWK3.txt', 'rb')
reg_PCNL_MAE_QWK3s = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_CEM.txt', 'rb')
reg_PCNL_MAE_CEMs = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_CEM2.txt', 'rb')
reg_PCNL_MAE_CEM2s = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_CEM3.txt', 'rb')
reg_PCNL_MAE_CEM3s = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_valid_preds.txt', 'rb')
reg_PCNL_MAE_valid_preds = pickle.load(f)
f.close()
f = open('reg_PCNL_MAE_valid_outputs_PCNL.txt', 'rb')
reg_PCNL_MAE_valid_outputs_PCNL = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_losses.txt', 'rb')
reg_PCNL_OAWE_losses = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_QWK.txt', 'rb')
reg_PCNL_MSE_QWKs = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_QWK2.txt', 'rb')
reg_PCNL_MSE_QWK2s = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_QWK3.txt', 'rb')
reg_PCNL_MSE_QWK3s = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_CEM.txt', 'rb')
reg_PCNL_MSE_CEMs = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_CEM2.txt', 'rb')
reg_PCNL_MSE_CEM2s = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_CEM3.txt', 'rb')
reg_PCNL_MSE_CEM3s = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_valid_preds.txt', 'rb')
reg_PCNL_MSE_valid_preds = pickle.load(f)
f.close()
f = open('reg_PCNL_MSE_valid_outputs_PCNL.txt', 'rb')
reg_PCNL_MSE_valid_outputs_PCNL = pickle.load(f)
f.close()

In [ ]:
f = open('oof_correct_ans.txt', 'rb')
oof_correct_ans = pickle.load(f)
f.close()

In [ ]:
CLASS = len(set(oof_correct_ans))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_class_OWAE_QWKs, reg_class_OWAE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_OWAE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_OWAE_valid_outputs, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_class_OWSE_QWKs, reg_class_OWSE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_OWSE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_OWSE_valid_outputs, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_PCNL_OWAE_QWKs, reg_PCNL_OWAE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_OWAE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_OWAE_valid_outputs_PCNL, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_PCNL_OWSE_QWKs, reg_PCNL_OWSE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_OWSE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_OWSE_valid_outputs_PCNL, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [multiclass_logloss_QWKs, multiclass_logloss_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(multiclass_logloss_valid_preds, best_epochs)]))
    if(name == 'QWK'):
        print('QWK :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if(name == 'CEM'):
        print('CEM :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_class_MAE_QWKs, reg_class_MAE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_MAE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_MAE_valid_outputs, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_class_MSE_QWKs, reg_class_MSE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_MSE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_class_MSE_valid_outputs, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        thresholds = [i+0.5 for i in range(CLASS-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_PCNL_MAE_QWKs, reg_PCNL_MAE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_MAE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_MAE_valid_outputs_PCNL, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))

In [ ]:
for name, scores in zip(['QWK', 'CEM'], [reg_PCNL_MSE_QWKs, reg_PCNL_MSE_CEMs]):
    best_epochs = np.argmax(np.array(scores), axis=1)
    oof_preds = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_MSE_valid_preds, best_epochs)]))
    oof_outputs = list(itertools.chain.from_iterable([valid_preds[best_epoch] for valid_preds, best_epoch in zip(reg_PCNL_MSE_valid_outputs_PCNL, best_epochs)]))
    if name == 'QWK':
        print(name, ':', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('QWK_after2 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'qwk')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x'])
        print('QWK_after3 :', cohen_kappa_score(oof_correct_ans, oof_preds, labels=None, weights='quadratic', sample_weight=None))
    if name == 'CEM':
        print(name, ':', CEM(oof_correct_ans, oof_preds))
        oof_preds = greedy_predict(oof_outputs, oof_correct_ans)
        print('CEM_after2 :', CEM(oof_correct_ans, oof_preds))
        pcnl = calc_PCNL(np.bincount(oof_correct_ans))
        thresholds = [(pcnl[i]+pcnl[i+1])/2 for i in range(len(pcnl)-1)]
        optR = OptimizedRounder(n_classes = CLASS, initial_coef = thresholds)
        optR.fit(oof_outputs, oof_correct_ans, loss_func = 'cem')
        oof_preds = optR.predict(oof_outputs, optR.coef_['x']).astype(int).tolist()
        print('CEM_after3 :', CEM(oof_correct_ans, oof_preds))